### **Paso 1.10 - Modificación final al proceso de ingesta desde HTTP**

#### Creamos el dataset **ds_ecdc_file_list**

En Azure Blob Storage creamos un nuevo contenedor **configs** y dentro de el tenemos el archivo **ecdc-file-list-for-2-files.json** con la siguiente estrcutura:
```
[
    {
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/cases_deaths.csv",
        "sinkFileName":"cases_deaths.csv"
    },
    {
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/hospital_admissions.csv",
        "sinkFileName":"hospital_admissions.csv"
    }
]
```
Crearemos un nuevo dataset llamado **ds_ecdc_file_list**

<center><img src="https://i.postimg.cc/Kzc0HWDt/adf737.png"></center>
<center><img src="https://i.postimg.cc/N0BDKnVZ/adf738.png"></center>

El dataset reutiliza el Linked Service que hace referencia hacia ABS **ls_ablob_covidreportingsa** y apunta hacia el contenedor **configs** y al archivo **ecdc-file-list-for-2-files.json**

<center><img src="https://i.postimg.cc/XYwx7W2w/adf739.png"></center>
<center><img src="https://i.postimg.cc/x8hgq1DP/adf740.png"></center>


#### Crearemos un pipeline llamado **test** para visualizar que nos devuelven las actividades de **Lookup**, **For each** y **Set variable**

En la actividad **Lookup** haremos uso del dataset **ds_ecdc_file_list**

<center><img src="https://i.postimg.cc/02KcFDDX/adf741.png"></center>

Ejecutamos utilizando **Debug** y revisamos que nos devuelve el **Output** de la actividad

<center><img src="https://i.postimg.cc/zBdxZkx6/adf742.png"></center>
<center><img src="https://i.postimg.cc/Y0hbpmV9/adf743.png"></center>

Agregaremos la actividad **For each** y en la pestaña **Settings**, para **Items** le pasamos como contenido dinámico lo obtenido en **value** por la actividad **Lookup**
```
@activity('Lookup1').output.value
```
<center><img src="https://i.postimg.cc/CMQccmBf/adf744.png"></center>

Creamos una variable llamada **sourceRelativeURL** a nivel de Pipeline

<center><img src="https://i.postimg.cc/y8qvHxT1/adf745.png"></center>

Modificamos la actividad de **For each**

<center><img src="https://i.postimg.cc/yNjPcPCx/adf746.png"></center>

Utilizamos una actividad **Set variable** y utilizamos la variable **sourceRelativeURL** a nivel de Pipeline y le pasamos como valor dinámico lo obtenido en **items** para **sourceRelativeURL**.
```
En la actividad "For each" --> Settings --> Items = items --> @activity('Lookup1').output.value

Y queremos que la variable "sourceRelativeURL" vaya tomando valores de manera dinámica, de acuerdo al valor que encuentre en el JSON para 'sourceRelativeURL'

[
    {
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/cases_deaths.csv",  <------------ Esto nos devuelve
        "sinkFileName":"cases_deaths.csv"
    },
    {
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/hospital_admissions.csv",  <------------ Esto nos devuelve
        "sinkFileName":"hospital_admissions.csv"
    }
]

De esta manera la variable "sourceRelativeURL" a nivel de Pipeline tomará dos valores

```
<center><img src="https://i.postimg.cc/bJF9FwJj/adf747.png"></center>

Ejecutamos utilizando **Debug** y revisamos que nos devuelve el **Output** del primer resultado de la actividad

<center><img src="https://i.postimg.cc/265xwZCw/adf748.png"></center>
<center><img src="https://i.postimg.cc/XqGkM6GB/adf749.png"></center>

Revisamos que nos devuelve el **Output** del segundo resultado de la actividad

<center><img src="https://i.postimg.cc/PJgzTVY3/adf750.png"></center>
<center><img src="https://i.postimg.cc/sfvkfB57/adf751.png"></center>

####

En Azure Blob Storage en el contenedor **configs** y subimos un nuevo archivo **ecdc-file-list.json** con la siguiente estrcutura:
```
[
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/cases_deaths.csv",
        "sinkFileName":"cases_deaths.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/hospital_admissions.csv",
        "sinkFileName":"hospital_admissions.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/testing.csv",
        "sinkFileName":"testing.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/country_response.csv",
        "sinkFileName":"country_response.csv"
    }
]
```

Hemos borrado el pipeline **test** creado anteriormente. Ahora editaremos el dataset **ds_ecdc_file_list** para que haga referencia al archivo recien subido **ecdc-file-list.json**

<center><img src="https://i.postimg.cc/hP448k3c/adf770.png"></center>

Luego, en el pipeline **pl_ingest_ecdc_data** añadiremos la actividad **Lookup**

<center><img src="https://i.postimg.cc/G2k3zdCh/adf763.png"></center>

Añadiremos también la actividad **For each** y en **Settings** en la opción de **Items** indicamos el siguiente contenido dinámico:
```
@activity('Lookup ECDC File List').output.value
```
<center><img src="https://i.postimg.cc/Xq4jzmjK/adf764.png"></center>

Configuramos la actividad **For each**, cortando y pegando dentro la actividad **Copy data** que teniamos en el pipeline 

<center><img src="https://i.postimg.cc/YChpZ1HP/adf765.png"></center>
<center><img src="https://i.postimg.cc/sxmjdmxk/adf766.png"></center>

Borramos todos los parámetros a nivel de pipeline, ya que, no los utilizaremos

<center><img src="https://i.postimg.cc/wTWqwwSJ/adf767.png"></center>

Para los parámetros del dataset de origen **ds_ecdc_raw_csv_http**: **relativeURL** y **baseURL**, le pasamos como valor dinámico lo obtenido en **items**, tanto para **sourceBaseURL**, **sourceRelativeURL** y **sinkFileName**.
```
En la actividad "For each" --> Settings --> Items = items --> @activity('Lookup ECDC File List').output.value

Y queremos que "sourceBaseURL", "sourceRelativeURL" y "sinkFileName" vayan tomando valores de manera dinámica, de acuerdo al valor que encuentre en el JSON:

[
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/cases_deaths.csv",
        "sinkFileName":"cases_deaths.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/hospital_admissions.csv",
        "sinkFileName":"hospital_admissions.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/testing.csv",
        "sinkFileName":"testing.csv"
    },
    {
        "sourceBaseURL":"https://github.com",
        "sourceRelativeURL":"cloudboxacademy/covid19/raw/main/ecdc_data/country_response.csv",
        "sinkFileName":"country_response.csv"
    }
]
```

<center><img src="https://i.postimg.cc/QM7jxhVk/adf768.png"></center>
<center><img src="https://i.postimg.cc/GmVcCH3m/adf769.png"></center>

**Validamos**  y ejecutamos utilizando **Debug** y podemos ver que los 4 archivos se han ingestado correctamente

<center><img src="https://i.postimg.cc/59SNC4gs/adf771.png"></center>
<center><img src="https://i.postimg.cc/26FmhGQ1/adf772.png"></center>

Finalmente, podemos crear un Trigger que ejecute esto temporalmente:

<center><img src="https://i.postimg.cc/cHN0v2nq/adf773.png"></center>
<center><img src="https://i.postimg.cc/rsfkYpYc/adf774.png"></center>
<center><img src="https://i.postimg.cc/rsVTJ4jQ/adf775.png"></center>
<center><img src="https://i.postimg.cc/G3KdpM3N/adf776.png"></center>
<center><img src="https://i.postimg.cc/vHSyrcVt/adf777.png"></center>
<center><img src="https://i.postimg.cc/zXf5HnVf/adf778.png"></center>
<center><img src="https://i.postimg.cc/vZgs3b7F/adf779.png"></center>